In [ ]:
import sys
import os
import json

REPO_PATH = os.path.abspath(os.path.join(os.getcwd(), '..'))
if REPO_PATH not in sys.path:
    sys.path.insert(0, REPO_PATH)

from src.llm import generate_response, model_llm
from src.classification import generate_response_classification
from src.hallucination import generate_response_hallucination


In [ ]:
MODELS_CONFIG = {
    "mistral-7b": { 
        "path": "../src/models/mistral-7b-instruct-v0.2.Q4_K_M.gguf", 
        "name": "Mistral 7B"
    },
    "qwen2.5-1.5b": { 
        "path": "../src/models/qwen2.5-1.5b-instruct-q8_0.gguf", 
        "name": "Qwen 2.5 1.5B"
    },
    "qwen2.5-7b": { 
        "path": "../src/models/qwen2.5-7b-instruct-q2_k.gguf", 
        "name": "Qwen 2.5 7B"
    }
}

list_models = [MODELS_CONFIG["mistral-7b"]["path"], MODELS_CONFIG["qwen2.5-1.5b"]["path"], MODELS_CONFIG["qwen2.5-7b"]["path"]]
list_name = [MODELS_CONFIG["mistral-7b"]["name"], MODELS_CONFIG["qwen2.5-1.5b"]["name"], MODELS_CONFIG["qwen2.5-7b"]["name"]]


with open('../data/data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

list_question = [item['question'] for item in data]
print(data)

[{'question': 'J’ai mal à la tête depuis deux jours, est-ce inquiétant ?', 'reponse': 'Les maux de tête sont des symptômes très fréquents dans la population générale. Ils peuvent être liés à de nombreux facteurs comme le stress, la fatigue, le manque de sommeil, la déshydratation ou encore la tension musculaire. Dans beaucoup de cas, ils sont temporaires et sans gravité. Les informations fournies ici sont générales et ont uniquement un but informatif et éducatif.', 'hallucination': 0, 'label': 0}, {'question': 'J’ai une douleur thoracique intense qui irradie dans le bras gauche', 'reponse': 'La douleur thoracique irradiant vers le bras gauche est décrite dans les informations médicales générales comme un symptôme pouvant être associé à des atteintes du cœur ou des vaisseaux. Ce type de symptôme est considéré comme potentiellement sérieux dans les descriptions de santé. Cette information est donnée de manière générale, sans établir de diagnostic.', 'hallucination': 0, 'label': 1}, {'que

We use the best model, Qwen 2.5 7B, to generate the answers and perform the classification because it achieved the best results in the previous evaluations. The code goes through each question, generates an answer, calculates the probability that the question requires an immediate teleconsultation, and checks if the answer is hallucinated. Depending on the probability, the message is adjusted to suggest either an immediate teleconsultation or more general advice. The results show that the model provides clear, coherent, and reliable answers, with good control over hallucinations, making it particularly suitable for this type of informative medical use.

In [3]:
for q in list_question:
    llm = model_llm(list_models[2])
    reponse = generate_response(llm, q)
    proba = float(generate_response_classification(llm, q))
    hallucination = generate_response_hallucination(llm, reponse)
    if proba >= 0.5:
        print(f"Question : {q}\nRéponse : {reponse + '\nSouhaitez-vous passer en téléconsultation maintenant ?'}\nProbabilité : {proba}\nHallucination : {hallucination}\n")
    else:
        print(f"Question : {q}\nRéponse : {reponse + '\nAu vu de vos symptômes, il n’est pas nécessaire de programmer une téléconsultation immédiatement.'}\nProbabilité : {proba}\nHallucination : {hallucination}\n")
    

Question : J’ai mal à la tête depuis deux jours, est-ce inquiétant ?
Réponse : Il est normal de se soucier quand on a mal à la tête, mais il est important de comprendre que les informations fournies ici sont de simples informations générales et non un avis médical. Voici quelques points à considérer :

1. La plupart des maux de tête sont temporaires et non alarmants. Ils peuvent être causés par la fatigue, la déshydratation, le stress ou certaines activités physiques.

2. Si vous avez des douleurs de tête qui sont récurrentes ou s'accompagnent de symptômes tels que la nausée, la vomissements, la sensibilité à la lumière ou la sensibilité au bruit, cela pourrait indiquer un type de maux de tête qui nécessite une attention supplémentaire.

3. Il est toujours bon de bien hydrater, de bien dormir et de maintenir un régime alimentaire équilibré pour aider à prévenir les maux de tête.

Si vous vous inquiétez ou si les maux de tête persistent, il est toujours conseillé de consulter un profess